# Case study: predictions and decisions for recommendation systems

This notebook is modified from https://github.com/Heewon-Hailey/multi-armed-bandits-for-recommendation-systems/tree/main

Predictions and recommendations based on the contextual information in system is an important and popular application. Multi-armed bandits (MABs) are a framework for sequential decision making under uncertainty. MABs solve problems in online advertising, information retrieval, and media recommendation. For instance, Yahoo! News decides what news items to recommend to users based on article content, user profile, and the historical engagement of the user with articles. Given decision making in this setting is sequential (what do we show next?) and feedback is only available for articles shown. MABs such as ɛ-Greedy and UCB show a perfect formulation. However, incorporating some element of user-article state requires contextual bandits: articles are arms; context per round incorporates information about both user and article (arm); and {0,1} -valued rewards represent clicks. Therefore the per round cumulative reward represents click-through-rate, which can maximise to drive user engagement and advertising revenue. 

## Recommendation systems for Bio-related areas:
Recommender systems have a wide range of applications in biology-related areas. Here are a few examples:

1. Drug discovery: Recommender systems can be used to predict the efficacy of drugs based on genomic and proteomic data.

2. Clinical decision support: Recommender systems can assist doctors in making treatment decisions by recommending personalized therapies based on the patient's genetic data.

3. Genomics research: Recommender systems can help researchers to identify relevant genes and genetic variants associated with specific diseases.

4. Ecology and conservation biology: Recommender systems can be used to predict the distribution of species based on environmental data.

5. Agricultural research: Recommender systems can be used to optimize crop yield by recommending optimal planting dates and crop varieties based on soil and weather data.

6. Microbiology: Recommender systems can assist in the identification of microbial communities in environmental samples based on genomic data.

7. Bioinformatics: Recommender systems can be used to suggest the most appropriate computational tools and methods for analyzing genomic and proteomic data.


## Datasets

In this notebook, we use the dataset `dataset.txt`, which contains 10,000 instances corrresponding to distinct site visits by users-events in the language of this part. Each instance comprises 102 space-delimited columns of integers:
 - Column 1: The arm played by a uniformly-random policy out of 10 arms (news articles)
 - Column 2: The reward received from the arm played|1 if the user clicked 0 otherwise; and
 - Columns 3-102: The 100-dim flattened context; 10 features per arm (incorporating the content of the article and its match with the visiting user), first the features for arm 1, then arm 2, etc. up to arm 10.


## Algorithms

* Decisions (bandits): ɛ-greedy MAB; UCB MAB
* Prediction and decision together: LinUCB contextual MAB including evaluation and hyperparameter tuning [1]
For evaluation, off-policy evaluation [1-2] is implemented.


## References 

[1] Lihong Li, Wei Chu, John Langford, Robert E. Schapire, ‘A Contextual-Bandit Approach to Personalized News Article Recommendation’, in Proceedings of the Nineteenth International Conference on World Wide Web (WWW’2010), Raleigh, NC, USA, 2010. 
https://arxiv.org/pdf/1003.0146.pdf

[2] Lihong Li, Wei Chu, John Langford, and Xuanhui Wang. ‘Unbiased offline evaluation of contextualbandit-based news article recommendation algorithms.’ In Proceedings of the Fourth ACM International Conference on Web Search and Data Mining (WSDM’2011), pp. 297-306. ACM, 2011.
https://arxiv.org/pdf/1003.5956.pdf


In [ ]:
# Do not edit. These are the only imports permitted.
import numpy as np
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt                   

## Some python background

In Python, a parent class is a class that another class, known as a child class, inherits from. The child class can inherit attributes and methods from the parent class, and can also add its own attributes and methods. This allows for code reuse and can simplify the development process.

An abstract method is a method declared in a parent class that does not have an implementation. Instead, the child class that inherits from the parent class must provide its own implementation of the abstract method. Abstract methods are useful for defining a common interface for a group of related classes, while allowing for each class to provide its own unique implementation of the method. In Python, an abstract method is defined using the `@abstractmethod` decorator, which indicates that the method is not meant to be called directly and must be implemented in a child class. Classes that contain abstract methods must also be marked as abstract using the `abc.ABC` or `abc.ABCMeta` metaclass.

In [ ]:
class MAB(ABC):
    """Base class for a contextual multi-armed bandit (MAB)
    
    Parameters
    ----------
    n_arms : int
        Number of arms.
    """
    # initialise and raise input errors
    def __init__(self, n_arms):
        if not type(n_arms)==int:
            raise TypeError("`n_arms` must be an integer")
        if not n_arms >= 0:
            raise ValueError("`n_arms` must be non-negative")
        self.n_arms = n_arms
        
    @abstractmethod
    # raise input errors
    def play(self, context):
        """Play a round
        
        Parameters
        ----------        
        context : float numpy.ndarray, shape (n_arms, n_dims), optional
            An array of context vectors presented to the MAB. The 0-th 
            axis indexes the arms, and the 1-st axis indexes the features.
            Non-contextual bandits accept a context of None.
        
        Returns
        -------
        arm : int
            Integer index of the arm played this round. Should be in the set 
            {0, ..., n_arms - 1}.
        """
        if not type(context) == np.ndarray:
            raise TypeError("`context` must be numpy.ndarray")
        if not context.shape == (n_arms, n_dims):
            raise TypeError("`context` must have shape (n_arms, n_dims)")
        self.context = context

    
    @abstractmethod
    # raise input errors
    def update(self, arm, reward, context):
        """Update the internal state of the MAB after a play
        
        Parameters
        ----------
        arm : int
            Integer index of the played arm in the set {0, ..., n_arms - 1}.
        
        reward : float
            Reward received from the arm.
        
        context : float numpy.ndarray, shape (n_arms, n_dims), optional
            An array of context vectors that was presented to the MAB. The 
            0-th axis indexes the arms, and the 1-st axis indexes the 
            features. Non-contextual bandits accept a context of None. 
        """
        if not (type(arm) == int or arm.dtype == 'int64'):
            raise TypeError("`arm` must be int type")
        if not (arm >= 0 and arm <= (n_arms-1)):
            raise ValueError("`arm` must be the the set {0, .., n_arms - 1}")
        if not (type(reward) == float or reward.dtype == 'float64'):
            raise TypeError("`reward` must be float type")
        if not (context.shape == (n_arms, n_dims) and context.dtype == 'float64') :
            raise TypeError("`context` must be float numpy in shape (n_events, n_arms, n_dims)")
        # get the values
        self.arm = arm
        self.reward = reward
        self.context = context



In [ ]:
# Define global functions 
def break_tie(_range):
    indices = np.argwhere(_range == np.max(_range))
    index = np.random.randint(0,len(indices))

    return indices[index][0]

## Load dataset

In [ ]:
# load dataset here

data = np.loadtxt("dataset.txt")
arms, rewards, contexts = data[:,0], data[:,1], data[:,2:]
arms = arms.astype(int)
rewards = rewards.astype(float)
contexts = contexts.astype(float)
n_arms = len(np.unique(arms))
n_events = len(contexts)
n_dims = int(len(contexts[0])/n_arms)
contexts = contexts.reshape(n_events, n_arms, n_dims)
    

In [ ]:
n_arms

In [ ]:
contexts.shape

## Off-policy evaluation

In the context of bandit problems, off-policy evaluation refers to the process of estimating the performance of a given policy using data collected under a different policy. In other words, we want to estimate how well a new policy would perform in the future based on historical data collected by a different policy. This is important because in many real-world scenarios, it may not be practical or ethical to experiment with a new policy on live users. Off-policy evaluation can be done using techniques such as importance sampling or weighted importance sampling. These methods re-weight the historical data to reflect the differences between the behavior policy and the target policy, allowing us to estimate the expected rewards that the target policy would have obtained if it had been used to collect the data.
Below we have a simplified version of offline evaluation.

In [ ]:
def offlineEvaluate(mab, arms, rewards, contexts, n_rounds=None):
    """Offline evaluation of a multi-armed bandit
    
    Parameters
    ----------
    mab : instance of MAB
        MAB to evaluate.
    
    arms : integer numpy.ndarray, shape (n_events,) 
        Array containing the history of pulled arms, represented as integer 
        indices in the set {0, ..., mab.n_arms}
    
    rewards : float numpy.ndarray, shape (n_events,)
        Array containing the history of rewards.
    
    contexts : float numpy.ndarray, shape (n_events, n_arms, n_dims)
        Array containing the history of contexts presented to the arms. 
        The 0-th axis indexes the events in the history, the 1-st axis 
        indexes the arms and the 2-nd axis indexed the features.
        
    n_rounds : int, default=None
        Number of matching events to evaluate the MAB on. If None, 
        continue evaluating until the historical events are exhausted.
    
    Returns
    -------
    out : float numpy.ndarray
        Rewards for the matching events.
    """
    # initialise values and raise input errors
    if not (arms.shape == (n_events,) and arms.dtype == 'int64')  :
        raise TypeError("`arms` must be integer numpy in shape (n_events,)")
    if not rewards.shape == (n_events,) and rewards.dtype == 'float64' :
        raise TypeError("`rewards` must be float numpy in shape (n_events,)")
    if not contexts.shape == (n_events,n_arms, n_dims) and rewards.dtype == 'float64' :
        raise TypeError("`contexts` must be float numpy in shape (n_events, n_arms, n_dims)")
    if n_rounds == None:        # set n_rounds to infinite number to run until all data exhausted
        n_rounds = np.inf
    elif not type(n_rounds) == int:
        raise TypeError("`n_rounds` must be integer or default 'None'")

    n_round = 0     # count the current round ; 0 indicates the first round
    R = []          # save the total payoff
    H = []          # save used historical events
    
    for i in range(n_events):
        if n_round == n_rounds:
            break
        arm = mab.play(contexts[i])
        if arm == arms[i]:                 # if historical data equals to chosen arm
            R.append(rewards[i])           # append the new rewards
            H.append([arms[i], rewards[i], contexts[i]])      # append the used events
            mab.update(arms[i], rewards[i], contexts[i])      # update the information
            n_round += 1

    # return rewards per play
    out = np.array(R)
        
    return out 

## 1. ε-greedy MAB

The epsilon-greedy algorithm is a popular approach in reinforcement learning and multi-armed bandit problems. It balances exploration and exploitation by selecting actions based on a trade-off between choosing the current best action (greedy) and exploring other actions (random). 

It works by assigning a small probability (epsilon) for random exploration, during which a random action is chosen, and a higher probability (1 - epsilon) for exploiting the current best action based on previous knowledge or learned values. This algorithm allows for the discovery of new potentially rewarding actions while still favoring actions with higher expected rewards based on the available information.

In [ ]:
class EpsGreedy(MAB):
    """Epsilon-Greedy multi-armed bandit

    Parameters
    ----------
    n_arms : int
        Number of arms

    epsilon : float
        Explore probability. Must be in the interval [0, 1].

    Q0 : float, default=np.inf
        Initial value for the arms.
    """
    # initialise values and raise input errors
    def __init__(self, n_arms, epsilon, Q0=np.inf):
        super().__init__(n_arms)
        if not (epsilon >= 0 and epsilon <= 1):
            raise ValueError("`epsilon` must be a number in [0,1]")
        if not type(epsilon) == float:
            raise TypeError("`epsilon` must be float")
        if not type(Q0) == float:
            raise TypeError("`Q0` must be a float number or default value 'np.inf'")
            
        self.epsilon = epsilon
        self.q = np.full(n_arms, Q0)      # initialise q values
        self.rewards = np.zeros(n_arms)     # keep the total rewards per arm
        self.clicks = np.zeros(n_arms)      # count the pulled rounds per arm
    
    # select a random arm to explore or a arm with best rewards to exploit, then return the arm 
    def play(self, context=None):
        super().play(context)
        
        # ------------------------------- #
        #       TODO: recommend arm       #
        arm = None
        #---------------------------------#
        return arm
    
    # update values
    def update(self, arm, reward, context=None):
        super().update(arm, reward, context)
        self.clicks[arm] += 1
        self.rewards[arm] += self.reward
        self.q[arm] = self.rewards[arm] / self.clicks[arm]
        

In [ ]:
mab = EpsGreedy(10, 0.05) 
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

## 2. Upper Confidence Bound (UCB)

The Upper Confidence Bound (UCB) algorithm is a strategy commonly used in multi-armed bandit problems to balance exploration and exploitation. It assigns each arm (action) a score based on its potential for high reward, taking into account both the average reward obtained from that arm and the uncertainty or variance associated with that estimate. The UCB algorithm selects the arm with the highest score at each step, prioritizing arms that have shown promise but have not been extensively explored. As more data is gathered, the algorithm becomes more confident in its estimates and gradually focuses on exploiting the arms with the highest potential for reward. The UCB algorithm offers a principled approach to optimize the trade-off between exploration and exploitation in decision-making tasks.

$$U C B_t(a)=\hat{\mu}_t(a)+\sqrt{\frac{\rho \log t}{T_t(a)}}$$

In [ ]:
class UCB(MAB):
    """Upper Confidence Bound (UCB) multi-armed bandit

    Parameters
    ----------
    n_arms : int
        Number of arms.

    rho : float
        Positive real explore-exploit parameter.

    Q0 : float, default=np.inf
        Initial value for the arms.
    """
    def __init__(self, n_arms, rho, Q0=np.inf):
        super().__init__(n_arms)
        if not rho > 0:
            raise ValueError("`rho` must be positive")
        if not (type(rho) == float and np.isreal(rho)):
            raise TypeError("`rho` must be real float")
        if not type(Q0) == float :
            raise TypeError("`Q0` must be a float number or default value 'np.inf'")
            
        self.rho = rho
        self.q = np.full(n_arms, Q0)
        self.rewards = np.zeros(n_arms)  
        self.avg_rewards = np.zeros(n_arms)
        self.clicks = np.zeros(n_arms)
        self.round = 0        # to count the number of round played
    
    def play(self, context=None):
        super().play(context)
        self.round += 1
        # ------------------------------- #
        #       TODO: calculate self.q    #
        self.q = None
        #---------------------------------#
        
        arm = break_tie(self.q)
        
        return int(arm)
        
    def update(self, arm, reward, context=None):
        super().update(arm, reward, context)
        self.clicks[arm] += 1
        self.rewards[arm] += reward
        self.avg_rewards[arm] = self.rewards[arm]/ self.clicks[arm]
        

In [ ]:
# soluation 
# self.q = np.where(self.clicks != 0, self.avg_rewards + np.sqrt(self.rho * np.log10(self.round) / self.clicks), self.q)


In [ ]:
# warning control
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
mab = UCB(10, 1.0)
results_UCB = offlineEvaluate(mab, arms, rewards, contexts,)
print('UCB average reward', np.mean(results_UCB))


## 3. (Contextual) LinUCB 

Now we put predictions and decision making together:

The LinUCB contextual algorithm is a variant of the Upper Confidence Bound (UCB) algorithm designed for contextual bandit problems. In contextual bandits, the algorithm receives additional contextual information or features along with each arm (action). The LinUCB algorithm models the relationship between the contextual features and rewards using linear regression. It maintains a set of linear models for each arm, estimating the expected reward based on the observed contextual information. The algorithm combines the estimated rewards with confidence bounds, computed using the uncertainty in the regression estimates, to make decisions about which arm to select. By incorporating contextual information and modeling the reward function through linear regression, the LinUCB algorithm aims to adaptively learn the optimal policy in contextual bandit settings, effectively balancing exploration and exploitation.

$$a_t \stackrel{\text { def }}{=} \arg \max _{a \in \mathcal{A}_t}\left(\mathbf{x}_{t, a}^{\top} \hat{\boldsymbol{\theta}}_a+\alpha \sqrt{\mathbf{x}_{t, a}^{\top} \mathbf{A}_a^{-1} \mathbf{x}_{t, a}}\right)$$
where $\mathbf{A}_a \stackrel{\text { def }}{=} \mathbf{D}_a^{\top} \mathbf{D}_a+\mathbf{I}_d$

In [ ]:
class LinUCB(MAB):
    """Contextual multi-armed bandit (LinUCB)

    Parameters
    ----------
    n_arms : int
        Number of arms.

    n_dims : int
        Number of features for each arm's context.

    alpha : float
        Positive real explore-exploit parameter.
    """
    # initialise values and raise input errors
    def __init__(self, n_arms, n_dims, alpha):
        if not (type(n_dims) == int or n_dims.dtype == 'int64'):
            raise TypeError("`n_dims` must be integer type")
        if not (type(alpha) == float or alpha.dtype == 'float64'):
            raise TypeError("`alpha` must be float")
        if not (alpha > 0.0 and np.isreal(alpha)):
            raise ValueError("`alpha` must be positive real")
        
        super().__init__(n_arms) 
        self.n_dims = n_dims
        self.alpha = alpha
        self.post_dist = np.zeros(n_dims)
        '''initialise keys and values; key is arm, A for covariance, inv_A for inverse of A, 
                                        b for reward, theta for coefficient vector''' 
        self.A = np.array(np.identity(n_dims))
        self.inv_A = [np.linalg.inv(self.A)]*10
        self.A  = [self.A]*10

        self.b = [np.zeros(n_dims)]*10
        self.theta = [(np.linalg.inv(np.identity(n_dims)) @  np.zeros(n_dims))]*10
         
    # return the best arm
    def play(self, context):
        super().play(context)
        # calculate posterior distribution of the coefficient vector 
        for arm in range(self.n_arms):
            inv_A = self.inv_A[arm]
            theta = self.theta[arm]

            
            # ------------------------------- #
            # TODO: calculate posterior distribution of the coefficient vector     
            self.post_dist[arm] = None
            #---------------------------------#
            
            
        arm = break_tie(self.post_dist)
        return int(arm)    
    
    # update dictionary
    def update(self, arm, reward, context):
        super().update(arm, reward, context)
        reshaped_context = context[arm].reshape(-1,1)   # reshape to the right form
        self.A[arm] = self.A[arm] + reshaped_context @ reshaped_context.T
        self.inv_A[arm] = np.linalg.inv(self.A[arm])
        self.b[arm] = self.b[arm] + reward * context[arm]
        self.theta[arm] = self.inv_A[arm] @ self.b[arm]
        
        
        

In [ ]:
# Solution
# self.post_dist[arm] = theta @ context[arm] + self.alpha * np.sqrt(context[arm].T @ inv_A @ context[arm])

In [ ]:
mab = LinUCB(10, 10, 1.0)
results_LinUCB = offlineEvaluate(mab, arms, rewards, contexts,800)
print('LinUCB average reward', np.mean(results_LinUCB))

## Evaluation 

In [ ]:
# plot cumulative reward per round 
eg_results, ucb_results, linucb_results, tbs_results, round_list = [], [], [], [], []  # create lists
n_rounds = 800                                       # the total number of rounds 
eg_sum, ucb_sum,linucb_sum, tbs_sum = 0, 0, 0, 0     # set the initial reward sum

# get the 800 results from the previous run per each algorithm 
for n_round in range(1, n_rounds + 1):              # start from 1 to avoid zero devision error
    eg_sum += results_EpsGreedy[n_round-1]
    ucb_sum += results_UCB[n_round-1]
    linucb_sum += results_LinUCB[n_round-1] 
    eg_results.append(eg_sum/n_round)
    ucb_results.append(ucb_sum/n_round)
    linucb_results.append(linucb_sum/n_round)
    round_list.append(n_round)

# plot the results
plt.plot(round_list,eg_results, label = "EpsGreedy")
plt.plot(round_list,ucb_results, label = "UCB")
plt.plot(round_list,linucb_results, label = "LinUCB")

plt.ylabel('Cumulative Reward Per-Round')
plt.xlabel('Rounds')
plt.legend()
plt.show()